Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [57]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [59]:
df.head(3)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
#importing libraries that i need
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#Data Wrangling to gather up the variables i'm using. Feature engineering


def wrangle(data):
  df = pd.read_csv(data,
                   parse_dates=['created'],
                   index_col ='created')
  
  #Added features
  df['TotAmeneties'] = (df['common_outdoor_space'] + df['wheelchair_access']
  + df['garden_patio'] + df['loft'] + df['exclusive']
  + df['terrace'] + df['swimming_pool'] + df['balcony']
  + df['high_speed_internet'] + df['outdoor_space']
  + df['roof_deck'] + df['laundry_in_unit'] + df['fitness_center']
  + df['laundry_in_building'] + df['dishwasher']
  + df['elevator'])
  df['TotalRooms'] = df['bedrooms'] + df['bathrooms']
  df['Cats&Dogs'] = 0
  df['CatsOrDogs'] = 0
  df.loc[(df['cats_allowed'] ==1) & (df['dogs_allowed'] ==1), 'Cats&Dogs'] = 1
  
  
  df = df[['price','Cats&Dogs', 'TotalRooms','TotAmeneties']]

  return df

In [110]:
df = wrangle(DATA_PATH+'apartments/renthop-nyc.csv')

df.head()

,price,Cats&Dogs,TotalRooms,TotAmeneties
created,,,,
2016-06-24 07:54:24,3000,0,4.5,0
2016-06-12 12:19:27,5465,1,3.0,2
2016-04-17 03:26:41,2850,0,2.0,2
2016-04-18 02:22:02,3275,0,2.0,0
2016-04-28 01:32:41,3350,0,5.0,0


In [112]:
#Splitting data
X = df[['TotAmeneties', 'Cats&Dogs', 'TotalRooms']]
y = df['price']
#Train and test splits
Apmay = df.loc['2016-04-01':'2016-05-31']
Jun = df.loc['2016-06-01':'2016-06-30']

X_train, y_train = Apmay[['TotAmeneties','Cats&Dogs',
                          'TotalRooms']], Apmay['price']
X_test, y_test = Jun[['TotAmeneties','Cats&Dogs',
                       'TotalRooms']], Jun['price']

In [113]:
#Establishing Baseline
print('Price mean', y_train.mean())
y_pred = [y_train.mean()] * len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Price mean 3734.503166915052
Baseline MAE: 1424.6919975334508


In [116]:
#Building the model

model = LinearRegression()
model.fit(X_train, y_train);

#Obtaining coefficients and intercept
print('model coeffiecients are:', model.coef_)
print('model intercept is:', model.intercept_)

model coeffiecients are: [135.54624305 327.4112467  922.65068088]
model intercept is: 762.6325171002386


In [121]:
#Checking for MAE
print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Test MAE:', mean_absolute_error(y_test, model.predict(X_test)))

Training MAE: 1068.3002755252044
Test MAE: 1338.4734641455366


In [123]:
#Checking for RMSE value (generalized price):
print('Training RMSE', mean_squared_error(y_train, model.predict(X_train), squared=False))
print('Test RMSE:', mean_squared_error(y_test, model.predict(X_test), squared=False))

Training RMSE 8670.474511810206
Test RMSE: 35416.086336864995


In [124]:
#Checking for R2 Values:
print('Training R^2:', r2_score(y_train, model.predict(X_train)))
print('Test R^2:', r2_score(y_test, model.predict(X_test)))

Training R^2: 0.026488270220629495
Test R^2: 0.0018363208632197203
